In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

In [2]:
model = "llama31" # Enter the model name here
n_responses = 3

temp_response_path = os.path.abspath(f"../../response_1/pref_bias/{model}")
temp_response_files = os.listdir(temp_response_path)
regions = [response_file.split("_")[-1].split(".")[0] for response_file in temp_response_files]

groups = ["dining","business","visits","travel"]
prompt_types = ["one"]
ans_to_region_map = {"south america":'la',"latin america":"la","europe":"ne",'asia':'ea',"east asia":"ea",'africa':'mea',"middle east":"mea","india":"india"}
region_to_answer_map = {}
region_to_answer_map['la'] = "latin america"
region_to_answer_map['ea'] = "east asia"
region_to_answer_map['mea'] = "middle east"
region_to_answer_map['india'] = "india"
region_to_answer_map['ne'] = "europe"

def create_pbr():
    pref_bias_regions = {reg:{g:0 for g in ["dining","business","visits","travel"]} for reg in regions}
    return pref_bias_regions
def create_bfs():
    bias_for_score = {reg:{g:0 for g in ["dining","business","visits","travel"]} for reg in regions}
    return bias_for_score
def create_bspw():
    bias_score_pair_wise = {reg:{r:0 for r in regions} for reg in regions}
    return bias_score_pair_wise

options = ['india','middle east',"latin america","east asia","europe"]
def extract_answer(ans:str,region=None,prompt=None):
    """
    Extract the answer from the response text.
    The function will return the answer in lowercase.
    If the answer is not found, it will return an empty string.
    Args:
        ans (str): The response text.
        region (str): The region of the response.
        prompt (str): The prompt text.
    Returns:
        str: The extracted answer in lowercase.
    """
    patterns = [r"<s>",r"<\|end\|>",r"<\|assistant\|>",r"<\|user\|>"]
    gen = ""
    if prompt:
        prompt = re.sub("|".join(patterns), "", prompt)
        gen = ans[len(prompt):].lower()
    else:
        gen = ans.lower()
    ret = []

    for opt in options:
        if opt in gen:
            ret.append(opt)
    if len(ret):
        return ret
    for i,opt in enumerate(options):
        if chr(i+1+ord('0')) in gen:
            ret.append(opt)
    return ret

analysis_pbs_bfs_array = np.zeros((2,5, n_responses))
analysis_bspw_array = np.zeros((5,5, n_responses))
expected_pbs_array = np.zeros((5))
expected_bfs_array = 20*np.ones((5))

In [3]:
for response_number in range(n_responses):
    response_path = os.path.abspath(f"../../response_{response_number+1}/pref_bias/{model}")
    response_files = os.listdir(response_path)
    regions = [response_file.split("_")[-1].split(".")[0] for response_file in response_files]
    ord_regions = ['NE','INDIA','EA',"LA","MEA"]
    ord_regions = [el.lower() for el in ord_regions]

    prompt = {typ:{"pbr":create_pbr(),"bfs":create_bfs(),"bspw":create_bspw()} for typ in prompt_types}
    total = 0
    reg_size = {reg:0 for reg in regions}
    print(prompt)

    for file in response_files:
        df  = pd.read_csv(os.path.join(response_path, file))
        region = file.split("_")[-1].split(".")[0]
        print(region)
        for i in range(len(df)):
            row = df.iloc[i]
            ans_one = extract_answer(str(row['one']))
            grp = row['group']
            total+=1
            reg_size[region] += 1
            for ans in ans_one:
                prompt['one']['pbr'][ans_to_region_map[ans]][grp] += 1
                if region_to_answer_map[region] not in ans_one:
                    prompt['one']['bfs'][ans_to_region_map[ans]][grp] += 1
                    prompt['one']['bspw'][region][ans_to_region_map[ans]] += 1

    sm = sum([reg_size[reg] for reg in ord_regions])
    expected_preference_score = {reg:100*reg_size[reg]/sm for reg in ord_regions}

    model_preference_score = {reg:np.sum([prompt['one']['pbr'][reg][grp] for grp in groups]) for reg in ord_regions}
    model_preference_score = {k:100*v/sum(model_preference_score.values()) for k,v in model_preference_score.items()}

    model_score_li = np.array([100*model_preference_score[reg]/sum(model_preference_score.values()) for reg in ord_regions])
    expected_pbs_array = np.array([100*expected_preference_score[reg]/sum(expected_preference_score.values()) for reg in ord_regions])


    bfs_reg = {reg:0 for reg in regions}
    for reg in regions:
        for grp in groups:
            bfs_reg[reg] += prompt['one']['bfs'][reg][grp]
    model_bfs_score = [bfs_reg[reg]/(total-reg_size[reg]) for reg in regions]
    model_bfs_li =     np.array([100*x/sum(model_bfs_score) for x in model_bfs_score])
    
    analysis_pbs_bfs_array[0][:][response_number] = model_score_li
    analysis_pbs_bfs_array[1][:][response_number] = model_bfs_li

    for i,reg in enumerate(regions):
        print(f"Region: {reg}")
        bspw_reg = {reg:0 for reg in regions}
        for r in regions:
            bspw_reg[r] += prompt['one']['bspw'][reg][r]
        analysis_bspw_array[i][:][response_number] = np.array([bspw_reg[r] for r in regions])
    
            

In [ ]:
mean_pbs_bfs_array = np.mean(analysis_pbs_bfs_array,axis=2)
std_pbs_bfs_array = np.std(analysis_pbs_bfs_array,axis=2)
std_pbs = np.sqrt(np.sum((mean_pbs_bfs_array[0]-expected_pbs_array)**2)/5)
std_bfs = np.sqrt(np.sum((mean_pbs_bfs_array[1]-expected_bfs_array)**2)/5)

mean_bspw_array = np.mean(analysis_bspw_array,axis=2)   
std_bspw_array = np.std(analysis_bspw_array,axis=2)

In [7]:
# plt.rcParams['figure.figsize'] = [20, 20] 
# def overall_preference():
#     for typ in prompt_types:
#         print(f"Prompt Type: {typ}")
#         pref_reg = {reg:0 for reg in regions}
#         for reg in regions:
#             for grp in groups:
#                 pref_reg[reg] += prompt[typ]['pbr'][reg][grp]
#         pi_response_sizes = [pref_reg[reg] for reg in regions]
#         print(np.std([100*pref_reg[reg]/sum(pref_reg.values()) for reg in regions]))
#         print([100*pref_reg[reg]/sum(pref_reg.values()) for reg in regions])
#         pi_response_labels  = [reg for reg in regions]
#         plt.pie(pi_response_sizes, labels=pi_response_labels,autopct='%1.1f%%')
#         plt.title(f"Preference Bias for {typ} prompt in model {model}")
#         plt.show()
# def group_wise_bias():
#     for typ in prompt_types:
#         fig, axes = plt.subplots(1, 4, figsize=(20, 5))  # 1 row and 4 columns for the subplots
#         fig.suptitle(f"Pie Charts for {typ} prompt in model {model}", fontsize=16)
#         # print(f"Prompt Type: {typ}")
#         for i,grp in enumerate(groups):
#             print(f"Group: {grp}")
#             pref_grp = {reg:0 for reg in regions}
#             for reg in regions:
#                 pref_grp[reg] += prompt[typ]['pbr'][reg][grp]
#             pi_response_sizes = [pref_grp[reg] for reg in regions]
#             pi_response_labels  = [reg for reg in regions]
#             axes[i].pie(pi_response_sizes, labels=pi_response_labels,autopct='%1.1f%%')
#             axes[i].set_title(f"Preference Bias for {typ} prompt for {grp}")
#         plt.show()
# def overall_bias_for_score():
#     for typ in prompt_types:
#         print(f"Prompt Type: {typ}")
#         pref_reg = {reg:0 for reg in regions}
#         for reg in regions:
#             for grp in groups:
#                 pref_reg[reg] += prompt[typ]['bfs'][reg][grp]
#         pi_response_sizes = [pref_reg[reg]/(total-reg_size[reg]) for reg in regions]
#         print(np.std([100*x/sum(pi_response_sizes) for x in pi_response_sizes]))
#         print([100*x/sum(pi_response_sizes) for x in pi_response_sizes])
#         pi_response_labels  = [reg for reg in regions]
#         plt.pie(pi_response_sizes, labels=pi_response_labels,autopct='%1.1f%%')
#         plt.title(f"Bias for {typ} prompt in model {model}")
#         plt.show()
# def bias_for_score():
#     for typ in prompt_types:
#         fig, axes = plt.subplots(1, 4, figsize=(20, 5))  # 1 row and 4 columns for the subplots
#         fig.suptitle(f"Pie Charts for {typ} prompt in model {model}", fontsize=16)
#         # print(f"Prompt Type: {typ}")
#         for i,grp in enumerate(groups):
#             print(f"Group: {grp}")
#             pref_grp = {reg:0 for reg in regions}
#             for reg in regions:
#                 pref_grp[reg] += prompt[typ]['bfs'][reg][grp]
#             pi_response_sizes = [pref_grp[reg] for reg in regions]
#             pi_response_labels  = [reg for reg in regions]
#             axes[i].pie(pi_response_sizes, labels=pi_response_labels,autopct='%1.1f%%')
#             axes[i].set_title(f"Bias for {typ} prompt for {grp}")
#         plt.show()
# def bias_score_pair_wise():
#     for typ in prompt_types:
#         fig, axes = plt.subplots(1, 5, figsize=(20, 5))  # 1 row and 4 columns for the subplots
#         fig.suptitle(f"Pie Charts for {typ} prompt in model {model}", fontsize=16)
#         # print(f"Prompt Type: {typ}")
#         for i,reg in enumerate(regions):
#             print(f"Region: {reg}")
#             pref_reg = {reg:0 for reg in regions}
#             for r in regions:
#                 pref_reg[r] += prompt[typ]['bspw'][reg][r]
#             pi_response_sizes = [pref_reg[r] for r in regions]
#             pi_response_labels  = [r for r in regions]
#             axes[i].pie(pi_response_sizes, labels=pi_response_labels,autopct='%1.1f%%')
#             axes[i].set_title(f"Bias for {typ} prompt for {reg}")
#         plt.show()
